In [ ]:
import pandas as pd

In [5]:
# Importing data

# We have 40 folders 'YYYY' (years 1980-2019)
# Each folder contains a 365*24 datasets with name 'YYYY_MM_DD_HH_UTC_County_Mean_Meteorology.csv'

# formula to calculate leap years
def leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)


# define parameters for data importing
YYYY = [f'{year:04}' for year in range(1980, 2020)]
DD = [f'{day:02}' for day in range(1, 32)]
HH = [f'{hour:02}' for hour in range(0, 24)]
MM = {f'{month:02}': DD if month not in {2, 4, 6, 9, 11} else DD[:30 if month != 2 else (29 if leap_year(year) else 28)] for month in range(1, 13)}

#define filename
names = []

for year in YYYY:
    for month in list(MM.keys()):
        for day in list(MM[month]):
            for hour in HH:
                names.append(f'/Users/G/Mentorship_dataset_Meteorology/historic/{year}/{year}_{month}_{day}_{hour}_UTC_County_Mean_Meteorology.csv')

In [ ]:
# import the data

df_list = []
for year in YYYY:
    for month in list(MM.keys()):
        for day in list(MM[month]):
            for hour in HH:
                # read the data
                df = pd.read_csv(f'/Users/G/Mentorship_dataset_Meteorology/historic/{year}/{year}_{month}_{day}_{hour}_UTC_County_Mean_Meteorology.csv')
                # keep the relevant columns only
                new_df = pd.DataFrame({
                    "date": [f"{year}-{month}-{day}-{hour}"] * len(df),
                    "year": [f"{year}"] * len(df),
                    "month": [f"{month}"] * len(df),
                    "T2": df["T2"],
                    "Q2": df["Q2"],
                    "U10": df["U10"],
                    "V10": df["V10"],
                    "SWDOWN": df["SWDOWN"],
                    "GLW": df["GLW"]
                })
                # add the new dframe to a list
                df_list.append(new_df)

In [ ]:
#These are 40*365*24 data frames
#We now average over the locations (FIPS) and create new dfs with columns T and Q

newlist_df= []
for i in range(0,len(df_list)):
    newlist_df.append(df_list[i].groupby(by=["date","month"]).agg(T=("T2","mean"),Q=("Q2","mean")).reset_index())
    
# concatenate the new datasets
final_df= pd.concat(newlist_df).reset_index()

In [ ]:
#We then export this df
final_df.to_csv('historical.csv')